In [21]:
# Importing all the packages
import pandas as pd
import numpy as np
import tensorflow
from tensorflow import keras as kr
import time

In [22]:
# Loading the datset
data = pd.read_csv('Concepts_9.csv')
data.head()

,Unnamed: 0,Response,Present_Difficulty,Time,Concept,Difficulty
0,0,1,1,16,1,2
1,1,1,1,29,1,1
2,2,1,1,37,1,1
3,3,1,1,36,1,1
4,4,1,1,23,1,2


In [29]:
# Seperating labels from features
all_types = data['Difficulty'].values
concepts = data['Concept'].values

In [31]:
# Encoding labels as one-hot vectors (categorical-integerlookup)
outputs_vals, outputs_ints = np.unique(all_types, return_inverse=True)
outputs_cats = kr.utils.to_categorical(outputs_ints)
# Encoding Concepts as one-hot vectors (categorical-integerlookup) Concepts-1 to 9
c_vals,c_ints = np.unique(concepts, return_inverse=True)
c_cats = kr.utils.to_categorical(c_ints)
# Appendinf the one hot vector of concepts to the input features array
inputs = data[['Response','Present_Difficulty', 'Time']].values
all_inputs  = np.append(inputs,c_cats,axis =1)


In [32]:
# Splitting the dataset into train and test
inds = np.random.permutation(len(all_inputs))
train_inds, test_inds = np.array_split(inds, 2)
inputs_train, outputs_train = all_inputs[train_inds], outputs_cats[train_inds]
inputs_test,  outputs_test  = all_inputs[test_inds],  outputs_cats[test_inds]

In [25]:
# Building Model
model = kr.models.Sequential()
model.add(kr.layers.Dense(32, input_shape=(4+8,)))
model.add(kr.layers.Activation("sigmoid"))
model.add(kr.layers.Dense(3))
model.add(kr.layers.Activation("softmax"))

In [33]:
# Training the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(inputs_train, outputs_train, epochs=100, batch_size=1, verbose=0)

In [34]:
# Printing the loss and accuracy
loss, accuracy = model.evaluate(inputs_test, outputs_test, verbose=0)
print("Loss: %10.5f\tAccuracy: %2.2f %%" % (loss, accuracy*100))

Loss:    0.12371	Accuracy: 94.81 %


In [38]:
# Predicting with custom sample
# Converting the concepts into one-hot-vectors
ints = [0,1,2,3,4,5,6,7,8]
c = kr.utils.to_categorical(ints)[3]
print(c)
# 1- Response, 2- Present Difficulty and 14- Time taken
test = [1,2,14]
test.extend(c) # adding the concept parameter to the test sample
#Predicting using the model
prediction = np.around(model.predict(np.expand_dims(test, axis=0))).astype(np.int)[0]  # not sure about the np.expand_dims command 
print("Model prediction in hot vector output: %s" % prediction)
print("Model prediction in actual Difficulty: %s" % outputs_vals[prediction.astype(np.bool)][0])


[0. 0. 0. 1. 0. 0. 0. 0. 0.]
Model prediction in hot vector output: [0 0 1]
Model prediction in actual Difficulty: 3


#  Testing with questions

In [10]:
#Loading questions dataset
q_data = pd.read_csv('MCQ_Dataset1.csv')
q_data.head()

,Qno,Questions,Correct,A,B,C,D,Concept,Difficulty
0,0,Knowing that Vijay's expenditure for first 3...,C,Rs. 220,Rs. 60,Rs. 50,Rs. 90,1.0,1.0
1,1,"Without considering the salary of the boss, t...",D,Rs. 30000,Rs. 27000,Rs. 36000,Rs. 29000,1.0,1.0
2,2,What will be average price of all the goods ...,B,Rs. 9,Rs. 6,Rs. 5.5,Rs. 8.25,1.0,1.0
3,3,How much will be the average of the squares o...,C,612.5,1225,426,324,1.0,1.0
4,4,Find average of natural numbers from 1 to 65?,A,33,32.5,130,65,1.0,1.0


In [11]:
#testing for concept-1
data_test = q_data

In [12]:
# Getting the all the attributes from one example in the dataset
q_no,Question,key,A,B,C,D,concept,Present_Difficulty= q_data.iloc[25]

In [15]:
#Question
# If the question is of Fill in the blank type - The options A,B,C and D are 0s in the dataset
# If the question is of MCQ type:

if(A!='0'):
 print("Q:",Question)
 print("A ",A)
 print("B ",B)
 print("C ",C)
 print("D ",D)
# If the question is of Fill in the  blank type type
else:
    print(Question)

Q:   Vijay says that his weight is between 75kg and 85kg. But his sister says that his weight is between 70kg and 80kg. his mother says that his weight cannot be greater than 78kg. If all the three predictions are correct, what is the average of different probable weights of Vijay?
A  75
B  80
C  78.5
D  76.5


In [39]:
#For user to answer and count the time using "time" library in python
begin = time.time()
Answer = input("Enter the option: ")
end = time.time()

Enter the option: D


In [17]:
#Converting concepts categories to one-hot vectors
ints = [0,1,2,3,4,5,6,7,8]
c = kr.utils.to_categorical(ints)[int(concept-1)]


In [40]:
# Predicting the difficulty of next question
if(A!='0'):
 if(Answer == key):
  response = 1
 else:
  response = 0
else:
  if(Answer == key):
    response = 1
  else:
    response = 0 
# Time taken converted to an integer value
time_taken = int(end-begin)
#Inputs to the model - response,Present_Difficulty,time_taken and concept as one-hot-vector
test = [response,Present_Difficulty,time_taken]
test.extend(c)
print("Response:",response," Time(s):",time_taken," Present Difficulty:",Present_Difficulty)
prediction = np.around(model.predict(np.expand_dims(test, axis=0))).astype(np.int)[0]
print("Model prediction in actual Difficulty: %s" % outputs_vals[prediction.astype(np.bool)][0])

Response: 1  Time(s): 8  Present Difficulty: 3.0
Model prediction in actual Difficulty: 3


# Using mySQL Table

In [19]:
# Importing necessary packages for mySQL server
import mysql.connector
import numpy
# Connceting to the server
try:
    connection = mysql.connector.connect(host='localhost',
                                         database='mydb', #Name of the database - mydb
                                         user='root',
                                         password='Root@123')
    # Name of the query - Questions
    sql_select_Query = "select * from questions"
    cursor = connection.cursor()
    cursor.execute(sql_select_Query)
    # get all records
    records = cursor.fetchall()
    print("Total number of rows in table: ", cursor.rowcount)
    #Storing each column in different arrays declared below
    qno = numpy.zeros(cursor.rowcount)
    question = numpy.empty(cursor.rowcount, dtype=object)
    ans = numpy.empty(cursor.rowcount, dtype=object)
    A = numpy.empty(cursor.rowcount, dtype=object)
    B = numpy.empty(cursor.rowcount, dtype=object)
    C = numpy.empty(cursor.rowcount, dtype=object)
    D = numpy.empty(cursor.rowcount, dtype=object)
    concept = numpy.zeros(cursor.rowcount)
    difficulty = numpy.zeros(cursor.rowcount)
    print("\nPrinting each row")
    i = 0
    for row in records:
        # Assigning each column to value to respective column names
        qno[i] = row[0]
        question[i] = row[1]
        ans[i] = row[2]
        A[i] = row[3]
        B[i] = row[4]
        C[i] = row[5]
        D[i] = row[6]
        concept[i] = row[7]
        difficulty[i] = row[8]
        i = i+1
        

except mysql.connector.Error as e:
    print("Error reading data from MySQL table", e)
finally:
    if connection.is_connected():
        connection.close()
        cursor.close()
        print("MySQL connection is closed")

ModuleNotFoundError: No module named 'mysql'

#  Writing the responses into the table

In [20]:
# Importing random package for selecting random question
import random
iter = 0
# A function to extract elements from list of list (as we get question numbers as list of list)
def Extract(lst):
    return [item[0] for item in lst]

# A function to select a random question with specified difficulty and concept from dataset of questions
def DIFF(concept, difficulty, con, diff):
 a = numpy.where(difficulty==diff)
 b = numpy.where(concept==con)
 c = a and b
 diff = []
 for i in range(30):
  diff.append([x[i] for x in c])
 d = Extract(diff)
 # selecting a rrandom question number
 return random.sample(d,1)


In [27]:
# To record the responses of the student in res2 list and difficulties as d1, question numbers Q1 (For sorting purpose according to question number) 
res1 = []
d1 = []
Q1 = []
# Begining with difficulty = 1 in concept = 1 
diff = 1
for i in range(5):
    # Getting a question number with given difficulty and concept    
    d1.append(diff)
    qno = DIFF(concept, difficulty, 1, diff)
    Q1.append(qno)
    q,k,a,b,c,d = question[qno],ans[qno],A[qno], B[qno], C[qno], D[qno]
    print("Q:",q)
    print("A",a)
    print("B",b)
    print("C",c)
    print("D",d)
    print("Key", k)
    begin = time.time()
    ans1 = input("Enter the option:")
    end = time.time()
    
    if(ans1 == k):
        response = 1
        res1.append(True)
    else:
        response =0
        res1.append(False)
    # Time taken converted to an integer value
    time_taken = int(end-begin)
    test = [response,diff,time_taken]
    ints = [0,1,2,3,4,5,6,7,8]
    c = kr.utils.to_categorical(ints)[int(1-1)]
    test.extend(c)
    print("Response:",response," Time(s):",time_taken," Present Difficulty:",diff)
    prediction = np.around(model.predict(np.expand_dims(test, axis=0))).astype(np.int)[0]
    print("Model prediction in actual Difficulty: %s" % outputs_vals[prediction.astype(np.bool)][0])
    diff = outputs_vals[prediction.astype(np.bool)][0] 

# The key is printed for testing purpose
# The difficulty of the next difficulty is predicted and the question with that difficulty is printed next

Q: ['  Average of six numbers comes out to be 3.95. Average of two of the numbers is 3.4 while the average of other two is 3.85. Determine the average of the two numbers left.']
A ['4.7']
B ['4.8']
C ['4.3']
D ['4.6']
Key ['D']
Enter the option:D
Response: 1  Time(s): 13  Present Difficulty: 1
Model prediction in actual Difficulty: 2
Q: ["  Knowing that Vijay's expenditure for first 3 days is Rs. 100, Rs. 125 and Rs. 85, what is his 4th day expenditure as his 4 days average expenditure Rs. 90?"]
A ['Rs. 220']
B ['Rs. 60']
C ['Rs. 50']
D ['Rs. 90']
Key ['C']
Enter the option:C
Response: 1  Time(s): 10  Present Difficulty: 2
Model prediction in actual Difficulty: 3
Q: ['  The average weight of three boys P, Q and R is 54 kg, while the average weight of three boys Q, S and T is 60 kg. What is the average weight of P, Q, R, S and T?']
A ['66.4 kg']
B ['63.2 kg']
C ['58.8 kg']
D ['Data Inadequate']
Key ['D']
Enter the option:A
Response: 0  Time(s): 15  Present Difficulty: 3
Model prediction

In [45]:
# Printing the question  umbers, difficulties, and the responses of the students
d1 = [ int(x) for x in d1]
Q1,d1,res1

([[29], [0], [22], [14], [20]],
 [1, 2, 3, 2, 1],
 [True, True, False, False, True])

In [29]:
# To record the responses of the student in res2 list and difficulty as d2, question numbers Q2 (For sorting purpose according to question number) 
res2 = []
d2 = []
Q2 = []
# Begining with difficulty = 1 in concept = 1 
diff = 1
for i in range(5):
    # Getting a question number with given difficulty and concept
    d2.append(diff)
    qno = DIFF(concept, difficulty, 1, diff)
    Q2.append(qno)
    q,k,a,b,c,d = question[qno],ans[qno],A[qno], B[qno], C[qno], D[qno]
    print("Q:",q)
    print("A",a)
    print("B",b)
    print("C",c)
    print("D",d)
    print("Key", k)
    begin = time.time()
    ans1 = input("Enter the option:")
    end = time.time()
    
    if(ans1 == k):
        response = 1
        res2.append(True)
    else:
        response =0
        res2.append(False)
    # Time taken converted to an integer value
    time_taken = int(end-begin)
    test = [response,diff,time_taken]
    ints = [0,1,2,3,4,5,6,7,8]
    c = kr.utils.to_categorical(ints)[int(1-1)]
    test.extend(c)
    print("Response:",response," Time(s):",time_taken," Present Difficulty:",diff)
    prediction = np.around(model.predict(np.expand_dims(test, axis=0))).astype(np.int)[0]
    print("Model prediction in actual Difficulty: %s" % outputs_vals[prediction.astype(np.bool)][0])
    diff = outputs_vals[prediction.astype(np.bool)][0] 

# The key is printed for testing purpose
# The difficulty of the next difficulty is predicted and the question with that difficulty is printed next

Q: ['  Of the 20 cycles sold by Ajay, average cost of 12 cycles is Rs. 18000. In total he earned Rs. 300000. What was the average cost of remaining cycles?']
A ['Rs. 10500']
B ['Rs. 7500']
C ['Rs. 9125']
D ['Rs. 9750']
Key ['A']
Enter the option:B
Response: 0  Time(s): 4  Present Difficulty: 1
Model prediction in actual Difficulty: 1
Q: ['  The average of first 17 multiples of 13 is']
A ['110.5']
B ['117']
C ['221']
D ['111']
Key ['B']
Enter the option:B
Response: 1  Time(s): 4  Present Difficulty: 1
Model prediction in actual Difficulty: 3
Q: ['  Average score for Virat Kohli in a series of 10 matches is 38.9 runs. If the average for first six matches comes out to be 42 what is his average in the last 4 matches of the series?']
A ['34.25']
B ['35']
C ['33.25']
D ['34.25']
Key ['A']
Enter the option:B
Response: 0  Time(s): 18  Present Difficulty: 3
Model prediction in actual Difficulty: 2
Q: ['  What will be average price of all the goods bought, if Ajay buys 30 erasers for Rs. 3 each,

In [50]:
# Printing the question  umbers, difficulties, and the responses of the students
d2 = [ int(x) for x in d2]
Q2,d2,res2

([[16], [19], [27], [2], [23]],
 [1, 1, 3, 2, 3],
 [False, True, False, True, False])

In [23]:
# For rearranging the responses with respect ot the question numbers
#qno1 = numpy.argsort(Q1)
#res1s = res1.copy()
#for i in range(len(qno1)):
#    res1s[i] = res1[qno1[i]]


In [31]:
# Writing the records to the responses table of mySQL table
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Root@123",
  database="mydb"
)

mycursor = mydb.cursor()
# Since tested with 5 questions, 5 responses are recorded
sql = "INSERT INTO responses VALUES (%s, %s, %s, %s, %s)"
val = [res1, res2]

mycursor.executemany(sql, val)

mydb.commit()

print(mycursor.rowcount, "was inserted.")

2 was inserted.


In [53]:
# Writing the records to the responses table of mySQL table
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Root@123",
  database="mydb"
)

mycursor = mydb.cursor()
# Since tested with 5 questions, 5 responses are recorded
sql = "INSERT INTO difficulties VALUES (%s, %s, %s, %s, %s)"
val = [d1,d2]
mycursor.executemany(sql, val)
mydb.commit()
print(mycursor.rowcount, "was inserted.")

2 was inserted.
